In [49]:
#imports
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import os 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [28]:
#loading dataset using panda 
#this loads one specific csv file
#dataset = pd.read_csv(r'C:\Users\benja\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data\ETFs\aadr.us.txt')
#dataset.head()
#recursively load all csv files into a list
#sets my working directory so the glob can fetch all the files
os.chdir(r'C:\Users\Admin\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data')
file_list = glob.glob('**/*.*',recursive=True)
"""""
dataset = pd.read_csv(file_list[0],delimiter=',',header=0)
print(dataset.info)
dataset2 = pd.read_csv(file_list[1],delimiter=',',header=0)
print(dataset2.info)
result = pd.concat([dataset.reset_index(drop=True),dataset2.reset_index(drop=True)])
print(result)
"""
dataset = pd.DataFrame()
#merges every dataframe in one big dataframe
"""""
for file in file_list:
    if(os.stat(file).st_size!=0):
        temp = pd.read_csv(file,sep=',',header=0)
        dataset = pd.concat([dataset.reset_index(drop=True),temp.reset_index(drop=True)])
print(dataset)
"""
#merges every dataframe in one big dataframe and check for empty files
for file in file_list:
    if(os.stat(file).st_size!=0):
        temp = pd.read_csv(file,sep=',',header=0)
        dataset = pd.concat([dataset.reset_index(drop=True),temp.reset_index(drop=True)])


Data Preprocessing

In [45]:

#order the dataset by date --> don't know if this is necessary
dataset = dataset.sort_values(by=['Date'])
#droping the date column --> not needed
dataset = dataset.drop(['Date'],axis=1)
#rescale the data to -1-1
scaler = MinMaxScaler(feature_range=(-1,1))
dataset = scaler.fit_transform(dataset)


In [10]:
#visualize the data

%load_ext tensorboard

Splitting data into training, test, validation set

In [46]:
#function for splitting the data since the predicted label is always the stock price for of the "next day"
def split_data(data,look_back):
    x = []
    y = []
    for i in range(len(data)-look_back-1):
        x.append(data[i:(i+look_back),:])
        y.append(data[i+look_back,:])
    return np.array(x),np.array(y)
def split_sets(x,y,x_split):
    x_train = x[:x_split]
    y_train = y[:x_split]
    x_test = x[x_split:]
    y_test = y[x_split:]
    return x_train,y_train,x_test,y_test


In [47]:
x,y = split_data(dataset,2)
#define training split --> 80% training, 10% testing, 10% validation
training_split = 0.8
#get the index where the data should be splitted
x_split = int(np.ceil(len(x)*training_split))
#split the data into training and testing and validation sets
x_train,y_train,x_test,y_test = split_sets(x,y,x_split)
#split the testing data into testing and validation sets with 70% testing and 30% validation
testing_split = 0.7
x_split = int(np.ceil(len(x_test)*testing_split))
x_test,y_test,x_val,y_val = split_sets(x_test,y_test,x_split)


Define model

In [48]:
#defining the model
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))
#compiling the model
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()



NameError: name 'LSTM' is not defined

train the model 

In [ ]:
#training the model
#overfitting on one sample
model.fit(x_train,y_train,validation_data=(x_val,y_val),batch_size=1,epochs=1)
